In [ ]:
# tại cell code này thì sẽ tạo ra submission chưa qua post-processing, điểm đạt được là 0.375
import pandas as pd
import numpy as np
import glob
import os

# --- CẤU HÌNH TRỌNG SỐ VÀ LỌC ---
W_SUBMISSION363 = 0.99
W_SUBMISSION261 = 0.01 
MIN_SCORE_THRESHOLD = 0.01 

print(f"-> Tỷ lệ: SUBMISSION363({W_SUBMISSION363}) + SUBMISSION261({W_SUBMISSION261})")

# 1. TÌM FILE
def find_file(filename):
    files = glob.glob(f'/kaggle/input/**/{filename}', recursive=True)
    if not files:
        raise FileNotFoundError(f"Lỗi: Không tìm thấy file '{filename}'")
    return files[0]

try:
    path_submission363 = find_file('ksub3.tsv')
    path_submission261 = find_file('lsub.tsv')
    
    print(f"Found SUBMISSION363: {path_submission363}")
    print(f"Found SUBMISSION261: {path_submission261}")
except Exception as e:
    print(f"LỖI: {e}")
    raise

# 2. ĐỌC DỮ LIỆU
dtypes = {'id': 'string', 'term': 'string'} 

df_submission363 = pd.read_csv(
    path_submission363,
    sep='\t',
    header=None,
    names=['id', 'term', 'score_submission363'],
    dtype=dtypes
)

df_submission261 = pd.read_csv(
    path_submission261,
    sep='\t',
    header=None,
    names=['id', 'term', 'score_submission261'],
    dtype=dtypes
)

# 3. TRỘN (MERGE)
print("-> Merging...")
df_blend = pd.merge(
    df_submission363,
    df_submission261,
    on=['id', 'term'],
    how='outer'
)

# 4. TÍNH ĐIỂM (DYNAMIC WEIGHTED AVERAGE)
print("-> Calculating Weighted Average...")

# ÉP KIỂU BẮT BUỘC
df_blend['score_submission363'] = pd.to_numeric(
    df_blend['score_submission363'], errors='coerce'
)
df_blend['score_submission261'] = pd.to_numeric(
    df_blend['score_submission261'], errors='coerce'
)

# Điền 0 vào chỗ thiếu
score_submission363_temp = df_blend['score_submission363'].fillna(0)
score_submission261_temp = df_blend['score_submission261'].fillna(0)

# TỬ SỐ
numerator = (
    score_submission363_temp * W_SUBMISSION363
    + score_submission261_temp * W_SUBMISSION261
)

# MẪU SỐ
weight_submission363_active = (
    df_blend['score_submission363'].notna().astype(float) * W_SUBMISSION363
)
weight_submission261_active = (
    df_blend['score_submission261'].notna().astype(float) * W_SUBMISSION261
)
denominator = weight_submission363_active + weight_submission261_active

# Xử lý chia cho 0 / NaN
df_blend['score'] = numerator / denominator
df_blend['score'] = df_blend['score'].fillna(0)

# 5. LỌC VÀ LƯU FILE
print("-> Filtering and Saving...")

# 5a. Lọc ngưỡng tối thiểu
df_final = df_blend[df_blend['score'] >= MIN_SCORE_THRESHOLD].copy()

# 5b. Loại bỏ trùng lặp
df_final.drop_duplicates(subset=['id', 'term'], keep='first', inplace=True)

# 5c. Sắp xếp theo ID
df_final = df_final.sort_values('id', ascending=True)

# 5d. Chọn cột, format và lưu
df_final = df_final[['id', 'term', 'score']]
df_final['score'] = df_final['score'].map('{:.3f}'.format)
df_final.to_csv('submission.tsv', sep='\t', header=False, index=False)

print(f"DONE! {len(df_final)} dòng.")


In [ ]:
#nâng điểm lên 0.378
import pandas as pd
from collections import defaultdict
import os
from tqdm import tqdm

# --- CẤU HÌNH ĐƯỜNG DẪN ---
INPUT_SUBMISSION = '/kaggle/input/merge-notebook/submission.tsv' #file cần xử lý
OBO_FILE = '/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo'
OUTPUT_FILE = 'submission.tsv'

# --- PHẦN 1: HÀM ĐỌC CẤU TRÚC OBO ---
def parse_go_obo(obo_path):
    print(f"Loading OBO from: {obo_path} ...")
    go_parents = defaultdict(set)
    current_id = None
    
    # Mở file và đọc từng dòng
    with open(obo_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line.startswith("id: GO:"):
                current_id = line.split("id: ")[1]
            elif line.startswith("is_a: GO:") and current_id:
                parent = line.split("is_a: ")[1].split(" !")[0]
                go_parents[current_id].add(parent)
            elif line.startswith("relationship: part_of GO:") and current_id:
                parent = line.split("relationship: part_of ")[1].split(" !")[0]
                go_parents[current_id].add(parent)
                
    print(f"-> Đã load xong quan hệ cha-con của {len(go_parents)} GO terms.")
    return go_parents

# --- PHẦN 2: HÀM TÌM TỔ TIÊN (ĐỆ QUY CÓ CACHE) ---
def get_ancestors(go_term, go_parents, cache):
    # Nếu đã tính rồi thì trả về ngay (Memoization)
    if go_term in cache:
        return cache[go_term]
    
    parents = set()
    # Lấy cha trực tiếp
    direct_parents = go_parents.get(go_term, [])
    
    for p in direct_parents:
        parents.add(p)
        parents.update(get_ancestors(p, go_parents, cache))
        
    cache[go_term] = parents
    return parents

# --- PHẦN 3: LOGIC LAN TRUYỀN (PROPAGATION) ---
def propagate_scores(df, go_parents):
    print("Bắt đầu lan truyền điểm số (Propagation)...")
    
    # Chuyển DataFrame thành Dictionary để xử lý cho nhanh
    # Cấu trúc: data[protein_id] = {go_term: score, ...}
    data = defaultdict(dict)
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Reading CSV"):
        data[row[0]][row[1]] = float(row[2]) # row[0]=ID, row[1]=Term, row[2]=Score
    
    final_rows = []
    ancestor_cache = {} # Cache để không phải tìm lại tổ tiên nhiều lần
    
    # Duyệt qua từng Protein
    for protein_id, terms_dict in tqdm(data.items(), desc="Propagating"):
        # Copy ra dict mới để chứa điểm sau khi lan truyền
        propagated_dict = terms_dict.copy()
        
        # Duyệt qua từng term hiện có của protein đó
        for go_term, original_score in terms_dict.items():
            # Lấy tất cả tổ tiên của term này
            ancestors = get_ancestors(go_term, go_parents, ancestor_cache)
            
            # Cập nhật điểm cho tổ tiên
            for ancestor in ancestors:
                current_score = propagated_dict.get(ancestor, 0.0)
                # Logic Max: Điểm tổ tiên = Max(Điểm cũ của nó, Điểm của con cháu)
                if original_score > current_score:
                    propagated_dict[ancestor] = original_score
        
        # Lưu lại kết quả vào list
        for term, score in propagated_dict.items():
            if score >= 0.001: # Lọc nhẹ bớt các điểm quá thấp để giảm dung lượng
                final_rows.append((protein_id, term, score))
                
    return final_rows

# --- MAIN ---
if __name__ == "__main__":
    # 1. Đọc dữ liệu
    if not os.path.exists(OBO_FILE):
        print("Lỗi")
        exit()
        
    go_parents = parse_go_obo(OBO_FILE)
    
    print(f"Reading submission: {INPUT_SUBMISSION}")
    df = pd.read_csv(INPUT_SUBMISSION, sep='\t', header=None, names=['Id', 'Term', 'Score'])
    
    # 2. Xử lý
    result_data = propagate_scores(df, go_parents)
    
    # 3. Xuất file
    print(f"Saving to {OUTPUT_FILE}...")
    result_df = pd.DataFrame(result_data, columns=['Id', 'Term', 'Score'])
    
    # Format float 3
    result_df.to_csv(OUTPUT_FILE, sep='\t', header=False, index=False, float_format='%.3f')